In [ ]:
from pyspark.sql import *
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pyspark.sql.functions as F
from IPython.display import display
from pyspark.sql.functions import to_timestamp
import pytz
from pyspark.sql.functions import *
import plotly.subplots as sp


In [ ]:
spark = SparkSession.builder.appName("test-app").getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType

# Define the explicit schema
schema = StructType([
    StructField("Timestamp", StringType(), nullable=True),
    StructField("Well_ID", StringType(), nullable=True),
    StructField("Stage", IntegerType(), nullable=True),
    StructField("Engine_Oil_Pressure", DoubleType(), nullable=True),
    StructField("Proppant_Concentration", DoubleType(), nullable=True),
    StructField("Fluid_Rate", DoubleType(), nullable=True),
])

# Read the CSV file with the specified schema
df = spark.read.csv("./TestData/oildata.csv", header=True, schema=schema)

# Create a temporary view
df.createOrReplaceTempView("data")

# Display the DataFrame
df.show()

In [ ]:
sql = """
SELECT Stage, AVG(Engine_Oil_Pressure) as avg_EOP, AVG(Fluid_Rate) as avg_FR
FROM data
GROUP BY Stage
"""
query = spark.sql(sql)
query.toPandas().sort_values(by=['Stage'])


In [ ]:
result_df=df.select('Engine_Oil_Pressure', 'Proppant_Concentration', 'Fluid_Rate', 'Well_ID','Stage')
result_df=result_df.toPandas()
result_df['Combined_Value'] = (result_df['Engine_Oil_Pressure'] * result_df['Fluid_Rate']) / result_df['Proppant_Concentration'] #moles*psi/liter
result_pivot = result_df.pivot_table(index='Well_ID', columns='Stage', values='Combined_Value', aggfunc='mean')
fig = px.imshow(result_pivot)
display(fig)


In [ ]:
data = df.toPandas()

# Get unique well IDs
unique_well_ids = data['Well_ID'].unique()

# Create a dictionary to map each well ID to a color
color_map = {well_id: color for well_id, color in zip(unique_well_ids, px.colors.qualitative.Pastel)}

# Assign colors based on well ID
data['Color'] = data['Well_ID'].map(color_map)

fig = go.Figure()
# Iterate over each unique well ID
for well_id in unique_well_ids:
    well_data = data[data['Well_ID'] == well_id]

    scatter_trace = go.Scatter(
        x=well_data['Timestamp'],
        y=well_data['Fluid_Rate'],
        mode='lines',
        name=f'Fluid Rate ({well_id})'
    )
    fig.add_trace(scatter_trace)
# Create a bar plot for Oil Pressure
bar_trace = go.Bar(
    x=data['Timestamp'], 
    y=data['Engine_Oil_Pressure'], 
    name='Engine Oil Pressure', 
    yaxis='y2',
    marker=dict(color=data['Color'])
)

# Create the figure and layout
fig = go.Figure(data=[scatter_trace, bar_trace])
fig.update_layout(
    title='Well Data',
    xaxis_title='Timestamp',
    yaxis=dict(title='Fluid Rate'),
    yaxis2=dict(title='Engine Oil Pressure', overlaying='y', side='right'),
    legend=dict(x=1, y=1),
    showlegend=True
)

display(fig)

In [ ]:
fig= px.line(data, x='Timestamp', y='Engine_Oil_Pressure',color='Well_ID')
display(fig)


In [111]:
sql = """
SELECT Stage, AVG(Engine_Oil_Pressure) as avg_EOP, AVG(Fluid_Rate) as avg_FR
FROM data
GROUP BY Stage
"""
query = spark.sql(sql)
data=query.toPandas().sort_values(by=['Stage'])

fig = go.Figure()
scatter_trace1=go.Scatter(
    x=data['Stage'],
    y=data['avg_FR'],
    mode='lines',
    name='Fluid Rate and Stage',
    marker=dict(color=data['Stage'])
)
scatter_trace2=go.Scatter(
    x=data['Stage'],
    y=data['avg_EOP'],
    mode='lines',
    name='Fluid Rate and Stage',
    marker=dict(color=data['Stage'])
)
fig = go.Figure(data=[scatter_trace1, scatter_trace2])
fig.update_layout(
    title='Well Data',
    xaxis_title='Stage',
    yaxis=dict(title='Fluid Rate'),
    yaxis2=dict(title='Engine Oil Pressure', overlaying='y', side='right'),
    legend=dict(x=1, y=1),
    showlegend=True
)
display(fig)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': array([1, 2, 3, 4], dtype=int32)},
              'mode': 'lines',
              'name': 'Fluid Rate and Stage',
              'type': 'scatter',
              'x': array([1, 2, 3, 4], dtype=int32),
              'y': array([550., 650., 730., 810.])},
             {'marker': {'color': array([1, 2, 3, 4], dtype=int32)},
              'mode': 'lines',
              'name': 'Fluid Rate and Stage',
              'type': 'scatter',
              'x': array([1, 2, 3, 4], dtype=int32),
              'y': array([100.93333333, 101.16666667,  99.9       , 100.9       ])}],
    'layout': {'legend': {'x': 1, 'y': 1},
               'showlegend': True,
               'template': '...',
               'title': {'text': 'Well Data'},
               'xaxis': {'title': {'text': 'Stage'}},
               'yaxis': {'title': {'text': 'Fluid Rate'}},
               'yaxis2': {'overlaying': 'y', 'side': 'right', 'title': {'text': 'Engine Oil Pressure'}}}
})